# Data Preparation

Created by Michael George (AKA Logiqx)

Link: https://www.speedsolving.com/forum/showthread.php?54128-How-fast-are-the-over-40-s-in-competitions

In [1]:
from EventsLib import *

## Data Prep

Turn the raw database extracts into a standard format:
* Split into multiple files - one file per event
* Standardise the layout - CSV with minimal quoting
* Apply time limits / cutoffs

Note: All of the output files can be made public due to the application of time limits / cutoffs

In [2]:
import os, csv

def prepareEvents(basename, subfolder, limit = True):
    """Split file into individual events and apply time limits"""
    
    def writeResults(basename, event, rows):
        """Write event results from memory to CSV"""

        fn = os.path.join('data', 'public', basename, event + '.csv')
        with open(fn, 'wb') as f:
            csvWriter = csv.writer(f, quoting = csv.QUOTE_MINIMAL)
            for row in rows:
                csvWriter.writerow(row)

    # Read rows using the CSV reader
    fn = os.path.join('data', 'private', subfolder, basename + '.csv')
    with open(fn, 'rb') as f:
        csvReader = csv.reader(f)
        
        # Initialisise the event results
        event = None
        results = []
        cutoff = 0
        count = 0
        
        # Process each row individually
        for inputRow in csvReader:

            # Only process the current row if it is a recognised event
            if inputRow[0] == event or eventsDict.has_key(inputRow[0]) > 0:
                
                # Has the event changed?
                if (event != inputRow[0]):
                    
                    # Save the previous event
                    if (event != None):
                        if count > 0:
                            results.append([cutoff, count])
                        writeResults(basename, event, results)
                    
                    # Initialisise the event results
                    event = inputRow[0]
                    results = []
                    cutoff = int(eventsDict[event][3])
                    count = 0                   
                    
                # Add the current row to the output buffer
                if (limit == False or int(inputRow[1]) < cutoff):
                    results.append(inputRow[1:])
                else:
                    count += int(inputRow[2])

        # Save the final event
        if (event != None):
            if count > 0:
                results.append([cutoff, count])
            writeResults(basename, event, results)

            
def preparePeople(basename, subfolder):
    """Essentially a file copy but it will reformat the records if necessary"""
    
    rows = []
    
    # Read rows using the CSV reader
    fn = os.path.join('data', 'private', subfolder, basename + '.csv')
    with open(fn, 'rb') as f:
        csvReader = csv.reader(f)
        for inputRow in csvReader:
            rows.append(inputRow)

    # Write rows using the CSV writer
    fn = os.path.join('data', 'public', basename + '.csv')
    with open(fn, 'wb') as f:
        csvWriter = csv.writer(f, quoting = csv.QUOTE_MINIMAL)
        for row in rows:
            csvWriter.writerow(row)

In [3]:
# Process data from database export
extract_date = '2019-01-30'
prepareEvents('partial_senior_averages', extract_date, limit = False)
prepareEvents('partial_senior_singles', extract_date, limit = False)
preparePeople('seniors', extract_date)
prepareEvents('known_senior_averages', extract_date)
prepareEvents('known_senior_singles', extract_date)
prepareEvents('wca_averages', extract_date)

# Process data from database export (provided by WCA results team)
extract_date = '2019-02-01'
prepareEvents('all_senior_averages', '2019-02-01')